In [134]:
import os
from datasets import Dataset
import soundfile as sf
import transformers
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Tokenizer, Wav2Vec2Processor
import torchaudio
import numpy as np
import torch
import traceback
from transformers import AutoProcessor
import multiprocessing

In [113]:
# # Define the feature extractor and tokenizer
# feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
# tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")

# # Define the processor
# processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")

ValueError: Unrecognized processor in facebook/hubert-large-ls960-ft. Should have a `processor_type` key in its preprocessor_config.json, or one of the following `model_type` keys in its config.json: clip, flava, layoutlmv2, layoutlmv3, layoutxlm, sew, sew-d, speech_to_text, speech_to_text_2, trocr, unispeech, unispeech-sat, vilt, vision-text-dual-encoder, wav2vec2, wav2vec2-conformer, wav2vec2_with_lm, wavlm

In [135]:
def load_data(root_dir):
    data = {"path": [], "transcript": [], "split": []}

    # For each subdirectory
    for split in ["Train", "Dev", "Test"]:
        split_dir = os.path.join(root_dir, split)
        for category in os.listdir(split_dir):
            category_dir = os.path.join(split_dir, category)

            # For each file
            for file in os.listdir(category_dir):
                if file.endswith(".flac"):
                    path = os.path.join(category_dir, file)
                    transcript_path = path.replace(".flac", ".txt")

                    with open(transcript_path, "r") as f:
                        transcript = f.read().strip()

                    data["path"].append(path)
                    data["transcript"].append(transcript)
                    data["split"].append(split)

    return Dataset.from_dict(data)


def map_to_array(batch):
    speech, _ = sf.read(batch["path"])
    batch["speech"] = speech
    return batch




In [136]:
# Load data
dataset = load_data("/raid/kaisar_dauletbek/datasets/ISSAI_KSC2")

# Set the number of processes to use for parallel processing
num_processes = multiprocessing.cpu_count()

# Parallelize the mapping operation across CPUs
dataset = dataset.map(map_to_array, num_proc=num_processes)

Exception ignored in: <function _releaseLock at 0x7faf7a35beb0>
Traceback (most recent call last):
  File "/home/kaisar_dauletbek/anaconda3/envs/nlp/lib/python3.10/logging/__init__.py", line 228, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


In [115]:
from datasets import load_dataset
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 22.62it/s]
                                                             

Dataset librispeech_asr_dummy downloaded and prepared to /home/kaisar_dauletbek/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_dummy/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b. Subsequent calls will reuse this data.


In [119]:
ds = ds.map(map_to_array)

In [120]:
ds

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id', 'speech'],
    num_rows: 73
})

In [37]:
# Play a flac file
file = '/raid/kaisar_dauletbek/datasets/ISSAI_KSC2/Train/crowdsourced/5f5310f5b7b74.flac'
speech, _ = sf.read(file)

In [39]:
import IPython
IPython.display.Audio(file)

In [40]:
import torch
import torchaudio
import transformers
import soundfile as sf
import numpy as np
from torch.utils.data import DataLoader
from transformers import Wav2Vec2CTCTokenizer, HubertConfig, HubertForCTC, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, TrainingArguments, Trainer
from datasets import Dataset, load_from_disk, load_metric
from torch.nn.utils.rnn import pad_sequence
import multiprocessing

2023-06-26 15:53:59.092485: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 15:53:59.956753: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-26 15:53:59.956826: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-06-26 15:53:59.956832: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [41]:
# Initialize a new tokenizer
tokenizer = Wav2Vec2CTCTokenizer("/raid/kaisar_dauletbek/Kazakh-Hubert/vocab.json", unk_token="<unk>", pad_token="<pad>", word_delimiter_token=" ")


# Train the tokenizer on your data
# tokenizer.train(files=dataset["path"])

# Initialize a new model configuration
config = HubertConfig()

# Initialize a new model with the configuration
# model = HubertModel(config)
model = HubertForCTC(config)

# Initialize a new feature extractor
feature_extractor = Wav2Vec2FeatureExtractor()

# Combine the tokenizer and feature extractor into a processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# def data_collator(features):
#     # Pad the input values
#     input_values = [feature["input_values"] for feature in features]
#     input_values = pad_sequence(torch.tensor(input_values), batch_first=True)

#     # Pad the labels
#     labels = [feature.get("labels", torch.tensor([-100])) for feature in features]
#     labels = pad_sequence(torch.tensor(labels), batch_first=True)

#     # Return the padded batch
#     return {"input_values": input_values, "labels": labels}

def data_collator(features):
    # Pad the input values
    input_values = [feature["input_values"] for feature in features]
    input_values = pad_sequence([torch.cat(input_value, dim=0) for input_value in input_values], batch_first=True)

    # Pad the labels
    labels = [feature.get("labels", torch.tensor([-100])) for feature in features]
    labels = pad_sequence([torch.cat(label, dim=0) for label in labels], batch_first=True)

    # Return the padded batch
    return {"input_values": input_values, "labels": labels}


data_collator = data_collator
wer_metric = load_metric("wer")

# Define the compute metrics function
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# dataloader = torch.load("/raid/kaisar_dauletbek/Kazakh-Hubert/dataloader.pt")
dataset = load_from_disk('/raid/kaisar_dauletbek/Kazakh-Hubert/dataset_ready')

# Define the data collator
# data_collator = DataCollatorCTC(processor, padding=True)

training_args = TrainingArguments(
    output_dir="/raid/kaisar_dauletbek/Kazakh-Hubert",
    group_by_length=True,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=30,
    fp16=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-4,
    warmup_steps=500,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset,
)


/tmp/ipykernel_2264487/926656413.py:34: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
Using cuda_amp half precision backend


In [42]:
dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'speech', 'input_values', 'labels'],
    num_rows: 18036
})

In [45]:
for row in dataset:
    print(torch.tensor(row["input_values"]).shape)

torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([206637])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([150272])
torch.Size([173655])
torch.Size([173655])
torch.Size([173655])
torch.Size([173655])
torch.Size([173655])
torch.Size([173655])


KeyboardInterrupt: 

In [43]:

print("Start training")
trainer.train()


The following columns in the training set don't have a corresponding argument in `HubertForCTC.forward` and have been ignored: speech, input_ids. If speech, input_ids are not expected by `HubertForCTC.forward`,  you can safely ignore this message.


Start training


KeyboardInterrupt: 